In [12]:
import pandas as pd

def parse_gedcom(file_path):
    individuals = {}
    current_individual = None
    current_individual_data = {}

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.startswith('0 @I'):
                if current_individual is not None:
                    individuals[current_individual] = current_individual_data
                    current_individual_data = {}
                current_individual = line.split('@')[1]
            elif line.startswith('1'):
                current_tag = line.split(' ')[1]
                current_individual_data[current_tag] = []
            elif line.startswith('2'):
                value = ' '.join(line.split(' ')[2:])
                current_individual_data[current_tag].append(value)
            else:
                continue

        if current_individual is not None:
            individuals[current_individual] = current_individual_data

    return individuals

individuals = parse_gedcom('BothHarmonSniderTree.ged')

individual_data = []
for individual_id, individual in individuals.items():
    data = {'ID': individual_id}
    for tag, values in individual.items():
        if tag == 'NAME':
            given_name, surname = values[0].split(' ', 1) if len(values) > 0 else ('', '')
            data['Name'] = ' '.join(values)
            data['Given_Name'] = given_name
            data['Surname'] = surname
        elif tag in ['BIRT', 'DEAT']:
            values_str = ' '.join(values)
            parts = values_str.split(' ', 1)
            if len(parts) > 1:
                date, place = parts
                data[f'{tag}_Date'] = date
                data[f'{tag}_Place'] = place
            else:
                data[f'{tag}_Date'] = values_str
                data[f'{tag}_Place'] = ''
        else:
            data[tag] = ' '.join(values)
    individual_data.append(data)

individual_df = pd.DataFrame(individual_data)

# Print the updated DataFrame
print(individual_df)

FileNotFoundError: [Errno 2] No such file or directory: 'example.ged'

In [2]:
import pandas as pd

# Assuming you have the `individual_df` DataFrame

# Export to Excel
individual_df.to_excel('individuals.xlsx', index=False)